<a href="https://colab.research.google.com/github/mehaase/w266-final-project/blob/main/DATASCI266_BERT_RandForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To read data back in:
from google.colab import drive
import pandas as pd

drive.mount('/mnt/gdrive')
project_dir = "/mnt/gdrive/MyDrive/W266 Final Project"
train_df = pd.read_csv(f"{project_dir}/train_df.csv", index_col=0)
test_df = pd.read_csv(f"{project_dir}/test_df.csv", index_col=0)

Mounted at /mnt/gdrive


In [ ]:
print(f"train_df: {len(train_df):,} rows")
print(f"test_df:  {len(test_df):,} rows")

train_df: 2,288 rows
test_df:  572 rows


**Random Forest with BERT Encoding Model**

In [ ]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.0 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, TFBertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import tensorflow as tf

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
bert_embeddings = bert_model.weights[0]
bert_embeddings

<tf.Variable 'tf_bert_model/bert/embeddings/word_embeddings/weight:0' shape=(30522, 768) dtype=float32, numpy=
array([[-0.01018257, -0.06154883, -0.02649689, ..., -0.01985357,
        -0.03720997, -0.00975152],
       [-0.01170495, -0.06002603, -0.03233192, ..., -0.01681456,
        -0.04009988, -0.0106634 ],
       [-0.01975381, -0.06273633, -0.03262176, ..., -0.01650258,
        -0.04198876, -0.00323178],
       ...,
       [-0.02176224, -0.0556396 , -0.01346345, ..., -0.00432698,
        -0.0151355 , -0.02489496],
       [-0.04617237, -0.05647721, -0.00192082, ...,  0.01568751,
        -0.01387033, -0.00945213],
       [ 0.00145601, -0.08208051, -0.01597912, ..., -0.00811687,
        -0.04746607,  0.07527421]], dtype=float32)>

In [ ]:
MAX_SEQUENCE_LENGTH = 128

#Tokenize tweet text with BERT and save as X_train, X_test
X_train_tokens = bert_tokenizer(
    train_df.tweet.tolist(),
    max_length=MAX_SEQUENCE_LENGTH,
    truncation=True,
    padding='max_length',
    return_tensors='tf',
)["input_ids"]
X_train_tensor = tf.nn.embedding_lookup(bert_embeddings, X_train_tokens)
X_train = tf.reshape(X_train_tensor, (X_train_tensor.shape[0], -1)).numpy()

X_test_tokens = bert_tokenizer(
    test_df.tweet.tolist(),
    max_length=MAX_SEQUENCE_LENGTH,
    truncation=True,
    padding='max_length',
    return_tensors='tf',
)["input_ids"]
X_test_tensor = tf.nn.embedding_lookup(bert_embeddings, X_test_tokens)
X_test = tf.reshape(X_test_tensor, (X_test_tensor.shape[0], -1)).numpy()

#Split train and test labels and convert to boolean
Y_train = train_df.is_hatespeech.astype(int)
Y_test = test_df.is_hatespeech.astype(int)

print("X_train tensor shape:", X_train_tensor.shape)
print("X_train numpy  shape:", X_train.shape)
print("X_test  tensor shape:", X_test_tensor.shape)
print("X_test  numpy  shape:", X_test.shape)

X_train tensor shape: (2288, 128, 768)
X_train numpy  shape: (2288, 98304)
X_test  tensor shape: (572, 128, 768)
X_test  numpy  shape: (572, 98304)


In [ ]:
def rf_model(X_train, Y_train, maxdepth = None, maxfeatures = 'sqrt', classweight = None):
  """
      Function for a baseline random forest model using default hyperparameters.
        Inputs
        X_train = embedded tweets
        Y_train = labels
        Output
        rf = trained random forest model

  """
  rf = RandomForestClassifier(max_depth = maxdepth, max_features = maxfeatures, class_weight = classweight)
  rf.fit(X_train, Y_train)

  return rf


In [ ]:
clf = rf_model(X_train, Y_train)

In [ ]:
rf_pred = clf.predict(X_train)
accuracy = accuracy_score(Y_train, rf_pred)
print("Train Accuracy:", accuracy)

rf_pred = clf.predict(X_test)
accuracy = accuracy_score(Y_test, rf_pred)
print("Test Accuracy :", accuracy)

print("\nTest metrics:")
precision, recall, f1, support = precision_recall_fscore_support(
    Y_test, rf_pred, average='binary')
print(f"Precision: {precision:0.4f}")
print(f"Recall:    {recall:0.4f}")
print(f"F1 Score:  {f1:00.4f}")

Train Accuracy: 1.0
Test Accuracy : 0.6451048951048951

Test metrics:
Precision: 0.6486
Recall:    0.6000
F1 Score:  0.6234


Using BERT tokenizer instead of bag of words tonizer approach is not beneficial to the test accuracy. We see a decrease of 0.1963 F-1 score when using BERT for tokenization.

# Hyperparameter Tuning For BERT + Random Forest

In [ ]:
#store metrics per hyperparameter tune
conditions = []
train_accuracies = []
test_accuracies = []
precision_scores = []
recall_scores = []
F1_scores = []

for maxdepth in [5, 10, 30]:
  for maxfeat in ['sqrt','log2',1000]:
    print("max_depth = ", maxdepth, "max_features: ", maxfeat)
    conditions.append(str(maxdepth) + "-" + str(maxfeat))

    clf = rf_model(X_train, Y_train, maxdepth = maxdepth, maxfeatures = maxfeat, classweight = 'balanced_subsample')
    rf_pred = clf.predict(X_train)
    accuracy = accuracy_score(Y_train, rf_pred)
    print("Train Accuracy:", accuracy)
    train_accuracies.append(accuracy)

    rf_pred = clf.predict(X_test)
    accuracy = accuracy_score(Y_test, rf_pred)
    print("Test Accuracy :", accuracy)
    test_accuracies.append(accuracy)

    precision, recall, f1, support = precision_recall_fscore_support(
        Y_test, rf_pred, average='binary')
    precision_scores.append(precision)
    recall_scores.append(recall)
    F1_scores.append(f1)

max_depth =  5 max_features:  sqrt
Train Accuracy: 0.7722902097902098
Test Accuracy : 0.5961538461538461

Test metrics:
max_depth =  5 max_features:  log2
Train Accuracy: 0.7412587412587412
Test Accuracy : 0.5926573426573427

Test metrics:
max_depth =  5 max_features:  1000
Train Accuracy: 0.7902097902097902
Test Accuracy : 0.6066433566433567

Test metrics:
max_depth =  10 max_features:  sqrt
Train Accuracy: 0.9816433566433567
Test Accuracy : 0.6486013986013986

Test metrics:
max_depth =  10 max_features:  log2
Train Accuracy: 0.9173951048951049
Test Accuracy : 0.6048951048951049

Test metrics:
max_depth =  10 max_features:  1000
Train Accuracy: 0.9847027972027972
Test Accuracy : 0.6468531468531469

Test metrics:
max_depth =  30 max_features:  sqrt
Train Accuracy: 1.0
Test Accuracy : 0.6293706293706294

Test metrics:
max_depth =  30 max_features:  log2
Train Accuracy: 1.0
Test Accuracy : 0.5909090909090909

Test metrics:
max_depth =  30 max_features:  1000
Train Accuracy: 1.0
Test Accu

In [ ]:
results_df = pd.DataFrame({'conditions':conditions})
results_df['train_accuracies'] = train_accuracies
results_df['test_accuracies'] = test_accuracies
results_df['precision_scores'] = precision_scores
results_df['recall_scores'] = recall_scores
results_df['F1_scores'] = F1_scores

In [ ]:
results_df

,conditions,train_accuracies,test_accuracies,precision_scores,recall_scores,F1_scores
0,5-sqrt,0.772290,0.596154,0.625641,0.435714,0.513684
1,5-log2,0.741259,0.592657,0.605381,0.482143,0.536779
2,5-1000,0.790210,0.606643,0.634146,0.464286,0.536082
3,10-sqrt,0.981643,0.648601,0.669528,0.557143,0.608187
4,10-log2,0.917395,0.604895,0.621622,0.492857,0.549801
5,10-1000,0.984703,0.646853,0.671053,0.546429,0.602362
6,30-sqrt,1.000000,0.629371,0.622302,0.617857,0.620072
7,30-log2,1.000000,0.590909,0.598291,0.500000,0.544747
8,30-1000,1.000000,0.613636,0.613027,0.571429,0.591497
